In [55]:
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import xarray as xr
import os
import numpy as np
import pandas as pd
import datetime 

In [56]:
# set variable downloaded from the 20th century Reanalysis Data
vrbl = 'RMM'

dataset_dir = '/mnt/beegfs/home/bommer1/WiOSTNN/Data/Index/'
    
start_yr, end_year = 1980, 2023 # set both to None to use all available years
months_to_keep = [11, 12, 1, 2, 3]
start_season, end_season = '11-10', '04-04'

file_name = 'rMII_index_latest.txt'
file_out = f"rmm_index_{start_yr}-{start_season}_{end_year}-{end_season}.nc"

In [57]:
# Load as numpy array.
X = np.loadtxt(dataset_dir + file_name)


## Format in dataset (xarray)

In [58]:
datetimes = []
rmm1 = []
rmm2 = []
indexs = []

for i in range(len(X)):
    datetimes.append(np.datetime64(datetime.date(int(X[i][0]), int(X[i][1]), int(X[i][2]))))
    rmm1.append(X[i][3])
    rmm2.append(X[i][4])
    indexs.append(X[i][5])


rmm1 = np.array(rmm1)
rmm2 = np.array(rmm2)
indexs = np.array(indexs)

In [59]:
ds = xr.Dataset(
    data_vars=dict(
        rmm1=(["time"], rmm1),
        rmm2=(["time"], rmm2),
        noaa_index = (["time"], indexs),
    ),
    coords=dict(
        time=datetimes,
    ),
    attrs=dict(description="RMM index from NOAA data updated 04.09.2023"),
)

In [60]:
ds_time = ds.sel({'time':slice(np.datetime64(f'{start_yr}-{start_season}'),np.datetime64(f'{end_year}-{end_season}'))})
ds_time.to_netcdf(dataset_dir + file_out)
print(ds_time)

<xarray.Dataset>
Dimensions:     (time: 15486)
Coordinates:
  * time        (time) datetime64[ns] 1980-11-10 1980-11-11 ... 2023-04-04
Data variables:
    rmm1        (time) float64 -1.04 -1.063 -1.053 -1.022 ... 0.0 0.0 0.0 0.0
    rmm2        (time) float64 -0.249 -0.414 -0.5852 -0.7159 ... 0.0 0.0 0.0 0.0
    noaa_index  (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 5.0 5.0 5.0 5.0 5.0
Attributes:
    description:  RMM index from NOAA data updated 04.09.2023


In [53]:
index_clc = xr.open_dataset(dataset_dir + 'mjo_rmm_index_noaa_1980-11-15_2023-03-31.nc')